<a href="https://colab.research.google.com/github/Vaibhav-S75/Team_404_Object_Detection/blob/main/Week_09_10k_images_withresults.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive to Access the Dataset
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Install Required Libraries
!pip install tensorflow keras opencv-python-headless matplotlib


In [ ]:
# Step 3: Set Paths for Dataset and Annotations
import os
import cv2
import numpy as np
import json

# Define your dataset paths
image_folder = '/content/drive/MyDrive/coco_data_set/train2014/train2014'  # Update with your path
annotations_path = '/content/drive/MyDrive/coco_data_set/annotations/annotations/instances_train2014.json'


In [ ]:
# Step 4: Load COCO Annotations
with open(annotations_path, 'r') as f:
    coco_annotations = json.load(f)

# Create a dictionary mapping image_id -> annotations
annotations_dict = {}
for annotation in coco_annotations['annotations']:
    image_id = annotation['image_id']
    if image_id not in annotations_dict:
        annotations_dict[image_id] = []
    annotations_dict[image_id].append(annotation)


In [ ]:
# Step 5: List Image Files and Filter Missing Images
all_files = [img for img in os.listdir(image_folder) if img.endswith(('.jpg', '.jpeg', '.png'))]

# Missing image IDs (replace this with your actual missing image IDs list)
missing_image_ids = [519138, 135420, 61830, 431980, 371598, 337844, 177109, 532482, 77693, 510622, ...]

# Filter out missing images and limit to the first 10,000 images
existing_images = []
image_ids = []
for image in all_files[:10000]:  # Limit to the first 10,000 images
    try:
        image_id = int(image.split('_')[-1].split('.')[0])
        if image_id not in missing_image_ids:
            existing_images.append(image)
            image_ids.append(image_id)
    except ValueError:
        print(f"Skipping file '{image}' due to ValueError.")

print(f"Total existing images after filtering: {len(existing_images)}")


Total existing images after filtering: 10000


In [ ]:
import gc  # For memory management

# Preprocessing function with reduced image resolution (128x128)
def preprocess_image(image_path, target_size=(128, 128)):  # Reduced resolution
    try:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            return None
        image_resized = cv2.resize(image, target_size)
        image_resized = image_resized.astype('float32') / 255.0
        return image_resized
    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

# Batch processing function with smaller batch size and memory management
def preprocess_images_in_batches(image_paths, batch_size=500):
    processed_images = []
    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]  # Select the current batch
        processed_batch = [preprocess_image(os.path.join(image_folder, img)) for img in batch]
        processed_batch = [img for img in processed_batch if img is not None]  # Filter out None values

        # Append processed batch to the main list
        processed_images.extend(processed_batch)
        print(f"Processed batch {i//batch_size + 1}: {len(processed_batch)} images")

        # Free memory after processing each batch
        del processed_batch  # Delete processed batch
        gc.collect()  # Explicitly call garbage collector to free memory

    return np.array(processed_images)

# Measure preprocessing time
import time
start_time = time.time()

# Call the function to preprocess images in batches
train_images = preprocess_images_in_batches(existing_images, batch_size=500)

end_time = time.time()
print(f"Total preprocessed images: {train_images.shape}")
print(f"Time taken for preprocessing: {end_time - start_time:.2f} seconds")


Processed batch 1: 500 images
Processed batch 2: 500 images
Processed batch 3: 500 images
Processed batch 4: 500 images
Processed batch 5: 500 images
Processed batch 6: 500 images
Processed batch 7: 500 images
Processed batch 8: 500 images
Processed batch 9: 500 images
Processed batch 10: 500 images
Processed batch 11: 500 images
Processed batch 12: 500 images
Processed batch 13: 500 images
Processed batch 14: 500 images
Processed batch 15: 500 images
Processed batch 16: 500 images
Processed batch 17: 500 images
Processed batch 18: 500 images
Processed batch 19: 500 images
Processed batch 20: 500 images
Total preprocessed images: (10000, 128, 128, 3)
Time taken for preprocessing: 812.76 seconds


In [ ]:
# Step 7: Get Bounding Boxes and Labels from Annotations
def get_annotations_for_images(image_ids, annotations_dict):
    all_boxes = []
    all_labels = []
    for image_id in image_ids:
        if image_id in annotations_dict:
            boxes = []
            labels = []
            for anno in annotations_dict[image_id]:
                x, y, width, height = anno['bbox']
                boxes.append([x, y, x + width, y + height])  # Convert to [x1, y1, x2, y2]
                labels.append(anno['category_id'])  # Add label
            all_boxes.append(boxes)
            all_labels.append(labels)
        else:
            all_boxes.append([])  # No annotations
            all_labels.append([])
    return all_boxes, all_labels

# Load bounding boxes and labels for preprocessed images
train_boxes, train_labels = get_annotations_for_images(image_ids, annotations_dict)
print(f"Example bounding boxes and labels for first image: {train_boxes[0]}, {train_labels[0]}")


Example bounding boxes and labels for first image: [[145.26, 107.87, 366.74, 516.31]], [87]


In [ ]:
# Step 8: One-hot Encode the Labels (Multi-label Binary Format)
def convert_labels_to_binary(labels, num_classes=80):
    binary_labels = np.zeros((len(labels), num_classes))
    for i, label_list in enumerate(labels):
        for label in label_list:
            # Ensure the label is within bounds
            if 1 <= label <= num_classes:
                binary_labels[i, label - 1] = 1  # COCO category_ids start at 1
            else:
                print(f"Warning: Label {label} is out of bounds for index {i}.")
    return binary_labels

# Convert train labels
train_labels_binary = convert_labels_to_binary(train_labels)


Streaming output truncated to the last 5000 lines.


In [ ]:
# Step 9: Split Data into Training and Validation Sets
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels_binary, val_labels_binary = train_test_split(
    train_images, train_labels_binary, test_size=0.2, random_state=42
)


In [ ]:
# Step 10: Define a Simplified CNN Model for Classification
from tensorflow.keras import layers, models

def create_simple_cnn_model(input_shape, num_classes=80):
    model = models.Sequential()
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))  # Reduced filter size
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))  # Reduced filters
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))  # Reduced dense layer size
    model.add(layers.Dense(num_classes, activation='sigmoid'))  # Multi-label classification
    return model


In [ ]:
# Step 11: Compile and Train the Simplified Model
input_shape = (128, 128, 3)  # Adjust based on your reduced image size
num_classes = 80  # COCO has 80 object categories

model = create_simple_cnn_model(input_shape, num_classes)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_images, train_labels_binary, epochs=5, batch_size=32, validation_data=(val_images, val_labels_binary))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 125s 494ms/step - accuracy: 0.6576 - loss: 0.1469 - val_accuracy: 0.5810 - val_loss: 0.1108
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 146s 511ms/step - accuracy: 0.6363 - loss: 0.1028 - val_accuracy: 0.6685 - val_loss: 0.1049
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 136s 487ms/step - accuracy: 0.6312 - loss: 0.0907 - val_accuracy: 0.6300 - val_loss: 0.1070
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 136s 463ms/step - accuracy: 0.5981 - loss: 0.0714 - val_accuracy: 0.5615 - val_loss: 0.1130
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 150s 496ms/step - accuracy: 0.5741 - loss: 0.0512 - val_accuracy: 0.4860 - val_loss: 0.1345


In [ ]:
# Step 12: Train the Model (with reduced dataset)
history = model.fit(train_images, train_labels_binary, epochs=5, batch_size=32, validation_data=(val_images, val_labels_binary))


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 122s 489ms/step - accuracy: 0.5423 - loss: 0.0300 - val_accuracy: 0.5350 - val_loss: 0.1601
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 133s 454ms/step - accuracy: 0.5532 - loss: 0.0157 - val_accuracy: 0.4695 - val_loss: 0.2061
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 142s 455ms/step - accuracy: 0.5019 - loss: 0.0079 - val_accuracy: 0.4100 - val_loss: 0.2417
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 117s 468ms/step - accuracy: 0.5050 - loss: 0.0038 - val_accuracy: 0.4790 - val_loss: 0.2871
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 141s 463ms/step - accuracy: 0.5194 - loss: 0.0025 - val_accuracy: 0.4935 - val_loss: 0.3102


In [ ]:
# Step 13: Check Accuracy and Evaluate the Model
val_loss, val_accuracy = model.evaluate(val_images, val_labels_binary)
val_accuracy_percentage = val_accuracy * 100  # Convert to percentage
print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy_percentage:.2f}%")


63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 140ms/step - accuracy: 0.4877 - loss: 0.3149
Validation Loss: 0.3102, Validation Accuracy: 49.35%


In [ ]:
# Step 14: Save the Model in the Recommended Format
model.save('object_detection_model_10000_images.keras')
